In [3]:

import json
import torch.utils
import torch.utils.data
from torchtune import config, modules, utils
from omegaconf import DictConfig
import torch
import pickle

import json
import argparse
import lm_eval
from lm_eval.models.huggingface import HFLM
from lm_eval.tasks import get_task_dict, TaskManager
from lm_eval.utils import make_table
from typing import Any, Dict, List, Tuple, Union
from torchtune.modules.tokenizers import ModelTokenizer
from torch.nn.utils.rnn import pad_sequence
import itertools
import json
import logging
import random
import time
from collections import defaultdict
from typing import TYPE_CHECKING, List, Optional, Union
import lm_eval.api.metrics
import lm_eval.api.registry
import lm_eval.models
from lm_eval.caching.cache import delete_cache
from lm_eval.evaluator_utils import (
    consolidate_results,
    get_sample_size,
    get_task_list,
    prepare_print_tasks,
    print_writeout,
    run_task_tests,
)
from lm_eval.loggers import EvaluationTracker
from lm_eval.loggers.utils import add_env_info, add_tokenizer_info, get_git_commit_hash
from lm_eval.tasks import TaskManager, get_task_dict
from lm_eval.utils import (
    eval_logger,
    handle_non_serializable,
    hash_string,
    positional_deprecated,
    simple_parse_args_string,
)
if TYPE_CHECKING:
    from lm_eval.api.model import LM
    from lm_eval.tasks import Task
import numpy as np

cfg_dict ={
    "data_pkl": "dump_data.pkl",
    "model":{"_component_": "torchtune.models.llama2.llama2_7b_mod"},
    "native_llama_path":  "./llama-2-7b-hf",
    "tokenizer":{
        "_component_": "torchtune.models.llama2.llama2_tokenizer",
        "path": "<native_llama_path>/tokenizer.model"
    },
    "checkpointer":{
        "_component_": "torchtune.utils.FullModelHFCheckpointer",
        # "checkpoint_dir": "./llama-2-7b-hf",
        "checkpoint_dir": "./exps/sr4_final/weights",
        "checkpoint_files": [
            # "pytorch_model-00001-of-00002.bin",
            # "pytorch_model-00002-of-00002.bin"
            "hf_model_0001_0.pt",
            "hf_model_0002_0.pt"
        ],
        "adapter_checkpoint": None,
        "recipe_checkpoint": None,
        "output_dir": "",
        "model_type": "LLAMA2"
    },
}